## Project: Collecting Delayed Air Travel from Domestic US Flights

### Author: Ben Farrell
### Date: 10/05/2020

## Table of Contents
<ol>
    <li><a href="#intro">Introduction</a></li>
    <li><a href="#lib">Import Libraries</a></li>
    <li><a href="#Gather">Gathering Data</a></li>
    <li><a href="#import">Import Into Pandas</a></li>
    <li><a href="#ref">References</a></li>  
</ol>

<a id='intro'></a>
## Introduction

Data was collected from [Bureau of Transport Logistics](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?pn=1)

Data will be downloaded programmatically from website using selenium webdriver. This will be in Zip file format. The zip files will then be unzipped, loaded in pandas and merged into one DataFrame.

<a id="lib"></a>
## Import Libraries

In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import os
from os import listdir
from pathlib import Path
import time
import math
import magic

<a id="Gather"></a>
## Gathering Data

- 1	Year	1987-2008
- 2	Month	1-12
- 3	DayofMonth	1-31
- 4	DayOfWeek	1 (Monday) - 7 (Sunday)
- 5	DepTime	actual departure time (local, hhmm)
- 6	CRSDepTime	scheduled departure time (local, hhmm)
- 7	ArrTime	actual arrival time (local, hhmm)
- 8	CRSArrTime	scheduled arrival time (local, hhmm)
- 9	UniqueCarrier	unique carrier code
- 10	FlightNum	flight number
- 11	TailNum	plane tail number
- 12	ActualElapsedTime	in minutes
- 13	CRSElapsedTime	in minutes
- 14	AirTime	in minutes
- 15	ArrDelay	arrival delay, in minutes
- 16	DepDelay	departure delay, in minutes
- 17	Origin	origin IATA airport code
- 18	Dest	destination IATA airport code
- 19	Distance	in miles
- 20	TaxiIn	taxi in time, in minutes
- 21	TaxiOut	taxi out time in minutes
- 22	Cancelled	was the flight cancelled?
- 23	CancellationCode	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
- 24	Diverted	1 = yes, 0 = no
- 25	CarrierDelay	in minutes
- 26	WeatherDelay	in minutes
- 27	NASDelay	in minutes
- 28	SecurityDelay	in minutes
- 29	LateAircraftDelay	in minutes'

In [2]:
#Data of interest
Fields = (['FlightDate',
           'Reporting_Airline',
           'Tail_Number',
           'OriginAirportID',
           'Origin',
           'OriginState',
           'DestAirportID',
           'Dest',
           'DestState',
           'DepTime',
           'CRSDepTime',
           'DepDelay',
           'CRSArrTime',
           'ArrTime',
           'ArrDelay',
           'Cancelled',
           'CancellationCode',
           'Diverted',
           'CRSElapsedTime',
           'ActualElapsedTime',
           'AirTime',
           'Distance',
           'CarrierDelay',
           'WeatherDelay',
           'NASDelay',
           'SecurityDelay',
           'LateAircraftDelay'
])

In [3]:
browser = webdriver.Chrome('/Users/benfarrell/Downloads/chromedriver')

In [4]:
url='https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236'

In [5]:
browser.get(url)

In [6]:
#Obtain form checkbox Titles
FieldNames = []
for i in range(122):
    if i%2 == 0:
        FieldNames.append(browser.find_elements_by_class_name('dataTD')[2:][i].text)

In [7]:
#Map Desired Data Titles to Form Titles, to allow CheckBox Marking by Index
mapping = {}
for k in Fields:
    try:
        mapping[k] = FieldNames.index(k)
    except Exception as e:
        print(f'Passed on {k}')
        pass

In [8]:
#untick all checkboxes on form
for checkbox in browser.find_elements_by_id('VarName'):
    if checkbox.is_selected():
        checkbox.click()
#Scroll to top of page
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)

In [9]:
#tick all Relevant Fields
for idx in mapping.values():
    browser.find_elements_by_id('VarName')[idx].click()
#Scroll to top of page
browser.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)

In [10]:
#Find Geography Dropdown
geo = Select(browser.find_element_by_id('GEOGRAPHY'))
#Set Geography to All - All 0 index
geo.select_by_index(0)

In [11]:
#Years to extract
yrs = [str(r) for r in range(2015,2020)]
#months = ['January','February','March','April','May','June',"July",'August','September','October','November','December']
months = [str(i) for i in range(12)]

In [12]:
for y in yrs:
    #Find Year Dropdown
    year = Select(browser.find_element_by_id('XYEAR'))
    #Set Year to 2019
    year.select_by_visible_text(y)
    #Allow sheet to update each year change
    time.sleep(5)
    for m in months:
        #Find Month Dropdown
        month = Select(browser.find_element_by_id('FREQUENCY'))
        #Set Month
        month.select_by_index(m)
        time.sleep(5)
        #Find Download Button
        dwl = browser.find_element_by_name('Download')
        dwl.click()
        #Allow form to update. Without this, the csv files risk duplicating previous months data
        time.sleep(30)

<a id="import"></a>
## Import Into Pandas

In [13]:
#List files in downloads - Input your download location
files = listdir('/Users/benfarrell/Downloads/')

In [14]:
#Create full file paths - Add your own location
fpath = ['/Users/benfarrell/Downloads/' + file for file in files]

In [15]:
#Sort files by datecreated
fpath.sort(key=os.path.getctime)

In [16]:
#Ignore irrelevant files
fpath = [d for d in fpath if d.split('.')[-1] == 'zip']
        

In [17]:
#Check list of files
fpath

['/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING.zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (1).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (2).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (3).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (4).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (5).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (6).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (7).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (8).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (9).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (10).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (11).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (12).zip',
 '/Users/benfarrell/Downloads/1016082797_T_ONTIME_REPORTING (13).zip',
 '/Users/benfarrell/

In [18]:
#Generate text string for months format MM 
ms = [str(i).rjust(2,'0') for i in range(1,13)]

In [19]:
#Unzip files
count = 0
for file in fpath:
    with ZipFile(file, 'r') as zipf:
        filename = zipf.namelist()[0]
        zipf.extract(filename)
    #Rename files to prevent further downloads overwritting, each CSV has the same name
    os.rename(filename,(yrs[math.floor(count/12)]+'_'+ms[count%12]+'.csv'))
    count += 1

In [20]:
#Create empty dataframe to append data to
df = pd.DataFrame()

In [21]:
#Loop through csv files
for y in ['2019']:
    for m in ms:
        try:
            csv_name = y+'_'+m+'.csv'
            ndf = pd.read_csv(csv_name)
            df = df.append(ndf,ignore_index=True)
        except Exception as e:
            try:
                #Catch exceptions where csv file is not encoded in UTF-8
                blob = open(csv_name,'rb').read()
                #Guess encoding using python-magic lib
                m = magic.Magic(mime_encoding=True)
                encod = m.from_buffer(blob)
                #Apply encoding
                ndf = pd.read_csv(csv_name,encoding=encod)
                df = df.append(ndf,ignore_index=True)
            except:
                #If still undecipherable, skip csv
                pass

/Users/benfarrell/anaconda3/envs/data/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


KeyboardInterrupt: 

In [30]:
#Save to CSV
df.to_csv('2015_2019_Flights.csv',index=False)

In [2]:
test = pd.read_csv('2015_2019_Flights.csv')

KeyboardInterrupt: 

<a id="ref"></a>
## References

[1] - [Bureau of Transport Logistics](https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp?pn=1)